<a href="https://colab.research.google.com/github/farhanfuadabir/Nurse-Care-Challenge-2020/blob/main/1D_CNN_Nurse_Care_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dependencies

In [ ]:
# cnn model
import numpy as np
import pandas as pd
from matplotlib import pyplot
from joblib import load, dump
from tqdm.notebook import tqdm
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

# Process Windowed Dataset for CNN

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
  dataframe = load(filepath)
  return dataframe.values
 
# load a list of files and return as a 3d numpy array
def load_group(filenames, path):
	loaded = list()
	for name in filenames:
		data = load_file(path + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = np.dstack(loaded)
	return loaded
 
# load a dataset group, such as train or test
def load_dataset_group(group, path):
	# load all 3 files as a single array
	filenames = ['acc_x_' + group + '.joblib', 'acc_y_' + group + '.joblib', 
              'acc_z_' + group + '.joblib']
	# load input data
	X = load_group(filenames, path)
	# load class output
	y = load_file(path + 'label_' + group + '.joblib')
	return X, y
 
# load the dataset, returns train and test X and y elements
def load_dataset(path):
  # load all train
  trainX, trainy = load_dataset_group('train', path)
  print('trainX shape: ', trainX.shape, '   |   trainy shape: ', trainy.shape)
  # load all test
  testX, testy = load_dataset_group('test', path)
  print('testX shape: ', testX.shape, '   |   testy shape: ', testy.shape)
  # zero-offset class values
  trainy = trainy - 1
  testy = testy - 1
  # one hot encode y
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)
  print('Final Dataset shape: \n\nTrain Set: ', end='')
  print(trainX.shape, '<----->', trainy.shape, '\nTest Set: ', testX.shape, '<----->', testy.shape)
  return trainX, trainy, testX, testy

In [ ]:
# load data
dataset_path = '/content/drive/MyDrive/NCC Train Test Set (Resampled 20Hz)/Split 80 20/Windowed 250/'
trainX, trainy, testX, testy = load_dataset(dataset_path)

trainX shape:  (93838, 250, 3)    |   trainy shape:  (93838,)
testX shape:  (22775, 250, 3)    |   testy shape:  (22775,)
Final Dataset shape: 

Train Set: (93838, 250, 3) <-----> (93838, 12) 
Test Set:  (22775, 250, 3) <-----> (22775, 12)


# CNN Model

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy, verbose, epochs, batch_size):
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  model = Sequential()
  model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
  model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
  model.add(Dropout(0.5))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics=['categorical_accuracy', f1_m, precision_m, recall_m])
  # fit network
  # model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # # evaluate model
  # _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  import time
  t_start = time.clock()
  history = model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

  t_end = time.clock()

  print('\n-------------------\n')

  #_, accuracy_train = model.evaluate(X_train, y_train)
  loss_train, accuracy_train, f1_score_train, precision_train, recall_train = model.evaluate(trainX, trainy, verbose=1)
  print('Train Accuracy: %.2f' % (accuracy_train*100))
  print('Train F1 Score: %.2f' % (f1_score_train*100))
  print('Train Time: ', t_end - t_start)

  t_start = time.clock()
  #_, accuracy_test = model.evaluate(X_test, y_test)
  loss_test, accuracy_test, f1_score_test, precision_test, recall_test = model.evaluate(testX, testy, verbose=1)
  print('Test Accuracy: %.2f' % (accuracy_test*100))
  print('Test F1 Score: %.2f' % (f1_score_test*100))
  t_end = time.clock()
  print('Test Time: ', t_end - t_start)

  return f1_score_test, history
 
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = np.mean(scores), np.std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 
# run an experiment
def run_experiment(repeats=10, verbose=False, epochs=10, batch_size=32):
  # repeat experiment
  scores = list()
  for r in tqdm(range(repeats)):
    score, history = evaluate_model(trainX, trainy, testX, testy, verbose, epochs, batch_size)
    print('Score:', '%.3f' % (score * 100.0))
    scores.append(score)
  # summarize results
  summarize_results(scores)

## Train

In [ ]:
# for single iteration
score, history = evaluate_model(trainX, trainy, testX, testy, verbose=True, epochs=100, batch_size=16)
print('Score:', '%.3f' % (score * 100.0))

Epoch 1/100
5865/5865 [==============================] - 27s 3ms/step - loss: 0.9834 - categorical_accuracy: 0.6366 - f1_m: 0.6021 - precision_m: 0.7540 - recall_m: 0.5102
Epoch 2/100
5865/5865 [==============================] - 19s 3ms/step - loss: 0.8413 - categorical_accuracy: 0.6740 - f1_m: 0.6524 - precision_m: 0.7789 - recall_m: 0.5679
Epoch 3/100
5865/5865 [==============================] - 20s 3ms/step - loss: 0.7847 - categorical_accuracy: 0.6956 - f1_m: 0.6788 - precision_m: 0.7879 - recall_m: 0.6019
Epoch 4/100
5865/5865 [==============================] - 19s 3ms/step - loss: 0.7570 - categorical_accuracy: 0.7012 - f1_m: 0.6894 - precision_m: 0.7872 - recall_m: 0.6189
Epoch 5/100
5865/5865 [==============================] - 19s 3ms/step - loss: 0.7225 - categorical_accuracy: 0.7164 - f1_m: 0.7068 - precision_m: 0.7926 - recall_m: 0.6425
Epoch 6/100
5865/5865 [==============================] - 19s 3ms/step - loss: 0.7037 - categorical_accuracy: 0.7261 - f1_m: 0.7220 - precisi




##### Epoch = 100, Batch Size = 32
```
6058/6058 [==============================] - 17s 3ms/step - loss: 0.6758 - categorical_accuracy: 0.7407 - f1_m: 0.6330 - precision_m: 0.6960 - recall_m: 0.6139
Train Accuracy: 74.07
Train F1 Score: 63.30
Train Time:  2592.538336
1507/1507 [==============================] - 4s 3ms/step - loss: 1.7214 - categorical_accuracy: 0.6988 - f1_m: 0.6712 - precision_m: 0.7069 - recall_m: 0.6567
Test Accuracy: 69.88
Test F1 Score: 67.12
Test Time:  5.332593999999972
```
##### Epoch = 100, Batch Size = 16
```
6058/6058 [==============================] - 16s 3ms/step - loss: 0.8060 - categorical_accuracy: 0.7124 - f1_m: 0.6629 - precision_m: 0.7169 - recall_m: 0.6431
Train Accuracy: 71.24
Train F1 Score: 66.29
Train Time:  5139.016927000001
1507/1507 [==============================] - 4s 3ms/step - loss: 1.3907 - categorical_accuracy: 0.6718 - f1_m: 0.6433 - precision_m: 0.6773 - recall_m: 0.6312
Test Accuracy: 67.18
Test F1 Score: 64.33
Test Time:  5.47839599999952
Score: 64.328
```


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['categorical_accuracy'])
#plt.plot(history.history['val_categorical_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# run the experiment for interations
run_experiment(10, verbose=False, epochs=40, batch_size=32)


-------------------

6058/6058 [==============================] - 15s 2ms/step - loss: 0.6889 - categorical_accuracy: 0.7361 - f1_m: 0.6779 - precision_m: 0.7449 - recall_m: 0.6560
Train Accuracy: 73.61
Train F1 Score: 67.79
Train Time:  748.2750169999999
1507/1507 [==============================] - 4s 2ms/step - loss: 1.2579 - categorical_accuracy: 0.6757 - f1_m: 0.6431 - precision_m: 0.6928 - recall_m: 0.6246
Test Accuracy: 67.57
Test F1 Score: 64.31
Test Time:  4.866907999999967
Score: 6430.920

-------------------

6058/6058 [==============================] - 15s 2ms/step - loss: 0.6836 - categorical_accuracy: 0.7371 - f1_m: 0.6850 - precision_m: 0.7489 - recall_m: 0.6637
Train Accuracy: 73.71
Train F1 Score: 68.50
Train Time:  745.265728
1507/1507 [==============================] - 4s 2ms/step - loss: 1.2347 - categorical_accuracy: 0.6818 - f1_m: 0.6520 - precision_m: 0.6941 - recall_m: 0.6330
Test Accuracy: 68.18
Test F1 Score: 65.20
Test Time:  4.843038999999862
Score: 6519.679

# Random Forest Model

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier

# load label set again
trainy_2 = load_file(dataset_path + 'label_train.joblib')
testy_2 = load_file(dataset_path + 'label_test.joblib')

clf = RandomForestClassifier(verbose=True,n_jobs=-1)

clf.fit(trainX, trainy_2)
y_pred = clf.predict(testX)

# Evaluate Algorithm
print("\n\nConfusion Matrix: \n\n",confusion_matrix(testy_2,y_pred))
print("\n\nReport: \n\n",classification_report(testy_2,y_pred))
print("Accuracy: ",accuracy_score(testy_2,y_pred))